In [2]:
import boto3

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

bucket = "sagemaker-{}-{}".format(region, account_id)
prefix = "demo-multimodel-endpoint"

role = "arn:aws:iam::617011600974:role/service-role/AmazonSageMaker-ExecutionRole-20210316T155032"



In [11]:
# Lets set up our SageMaker notebook instance for local mode.
#!/bin/bash ./utils/setup.sh

The user has root access.
Error: argument "agent" is wrong: table id value is invalid



In [7]:
from time import gmtime, strftime

model_name = "MONAI-DEMO-MultiModelModel" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "s3://sagemaker-eu-west-1-617011600974/monai-on-sagemaker-docker-2022-06-03-08-24-03-985/output/model.tar.gz"

model_url = "s3://sagemaker-eu-west-1-617011600974/monai-on-sagemaker-docker-2022-06-03-08-24-03-985/output"


container = "617011600974.dkr.ecr.eu-west-1.amazonaws.com/monai-braintumor-segmentation:latest"

print("Model name: " + model_name)
print("Model data Url: " + model_url)
print("Container image: " + container)

container = {"Image": container, "ModelDataUrl": model_url, "Mode": "MultiModel"}

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, Containers=[container]
)

print("Model Arn: " + create_model_response["ModelArn"])


Model name: MONAI-DEMO-MultiModelModel2022-06-08-15-54-29
Model data Url: s3://sagemaker-eu-west-1-617011600974/monai-on-sagemaker-docker-2022-06-03-08-24-03-985/output
Container image: 617011600974.dkr.ecr.eu-west-1.amazonaws.com/monai-braintumor-segmentation:latest
Model Arn: arn:aws:sagemaker:eu-west-1:617011600974:model/monai-demo-multimodelmodel2022-06-08-15-54-29


In [8]:
endpoint_config_name = "MONAI-DEMO-MultiModelEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint config name: " + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Endpoint config name: MONAI-DEMO-MultiModelEndpointConfig-2022-06-08-15-54-46


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: MultiModel mode is not supported for instance type ml.g4dn.xlarge.

In [5]:
import time

endpoint_name = "MONAI-DEMO-MultiModelEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint name: " + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Endpoint Status: " + status)

print("Waiting for {} endpoint to be in service...".format(endpoint_name))
waiter = sm_client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=endpoint_name)

Endpoint name: MONAI-DEMO-MultiModelEndpoint-2022-06-08-15-41-56
Endpoint Arn: arn:aws:sagemaker:eu-west-1:617011600974:endpoint/monai-demo-multimodelendpoint-2022-06-08-15-41-56
Endpoint Status: Creating
Waiting for MONAI-DEMO-MultiModelEndpoint-2022-06-08-15-41-56 endpoint to be in service...


In [6]:
%%time

import json

payload = "Test Data"

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/x-image",
    TargetModel=".tar.gz",  # this is the rest of the S3 path where the model artifacts are located
    Body=payload,
)

print(*json.loads(response["Body"].read()), sep="\n")

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Failed to download model data(bucket: sagemaker-eu-west-1-617011600974, key: monai-on-sagemaker-docker-2022-06-03-08-24-03-985/output/model.tar.gz.tar.gz). Please ensure that there is an object located at the URL and that the role passed to CreateModel has permissions to download the model.
